# Bias Visualization

### Dependencies
Note that we use the scikit-learn confusion matrix utility to help calculate some of the fairness metrics. We generate the visualizations using plotly Express.

In [81]:
%%capture
%pip install kaleido numpy pandas plotly scikit-learn tqdm ipywidgets

In [82]:
import os
from math import sqrt
from typing import Any, Dict, List

import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.express as px
from IPython.display import clear_output, display
from sklearn.metrics import confusion_matrix as ConfusionMatrix
from tqdm.auto import tqdm

### Global Constants

This notebook will visualize how each of the following variable affects the fairness towards a particular protected group.

In [83]:
INDEPENDENT_VARIABLES = ["model", "dataset", "num_params"]

## Utilities

### Plotting Utils

In [84]:
def format_metric_name(metric_name: str) -> str:
    """
    Prettify diagram texts by replacing
    snake case with regular title case.
    """
    output_words = []
    for word in metric_name.split("_"):
        output_words.append(word.title())

    return " ".join(output_words)

## Getting Predictions Ready

### Load Prediction Files
Load all prediction `tsv` files from a given folder into a dataframe.

In [85]:
# prediction_folder: str = "resources/race_predictions/"
# prediction_files: List[str] = os.listdir(prediction_folder)
# prediction_tsv_paths: List[str] = [
#     os.path.join(prediction_folder, prediction_file)
#     for prediction_file in prediction_files
#     if prediction_file.endswith(".tsv") and "placeholder" not in prediction_file
# ]

# case_name = "amazon_roberta-base-SST5_predictions"
# case_name = "amazon_roberta-base-TweetEval_predictions"
# case_name = "NS-prompts_roberta-base-SST5_predictions"
# case_name = "NS-prompts_roberta-base-TweetEval_predictions"
# case_name = "regard_roberta-base-SST5_predictions"
# case_name = "regard_roberta-base-TweetEval_predictions"


# case_name = "amazon_opt-125m_SST5"
# case_name = "amazon_opt-350m_SST5"
# case_name = "amazon_roberta-base_SST5"
# case_name = "amazon_roberta-large_SST5"
plot_name = "amazon"
case_names = ["amazon_opt-125m_SST5", "amazon_opt-350m_SST5", "amazon_roberta-base_SST5","amazon_roberta-large_SST5", "amazon_llama-7b_SST5"]

# case_name = "NS-prompts_opt-125m_SST5"
# case_name = "NS-prompts_opt-350m_SST5"
# case_name = "NS-prompts_roberta-base_SST5"
# case_name = "NS-prompts_roberta-large_SST5"
# plot_name = "NS-prompts"
# case_names = ["NS-prompts_opt-125m_SST5", "NS-prompts_opt-350m_SST5", "NS-prompts_roberta-base_SST5", "NS-prompts_roberta-large_SST5", "NS-prompts_llama-7b_SST5"]

# case_name = "regard_opt-125m_SST5"
# case_name = "regard_opt-350m_SST5"
# case_name = "regard_roberta-base_SST5"
# case_name = "regard_roberta-large_SST5"
# plot_name = "regard"
# case_names = ["regard_opt-125m_SST5", "regard_opt-350m_SST5", "regard_roberta-base_SST5", "regard_roberta-large_SST5", "regard_llama-7b_SST5"]


prediction_tsv_paths = [
    "resources/predictions_all/" + case_name + ".tsv" for case_name in case_names
]

print(prediction_tsv_paths)

plot_dir = "plots/" + plot_name + "/"
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

output_folder: str = "stats"
output_table = pd.DataFrame()

for prediction_tsv_path in tqdm(prediction_tsv_paths, ncols=80):
    prediction_tsv_filename = os.path.basename(prediction_tsv_path)
    dataframe = pd.read_csv(prediction_tsv_path, delimiter="\t")
    
    del dataframe["text"]

    if output_table is None:
        output_table = dataframe
    else:
        assert isinstance(output_table, pd.DataFrame)
        output_table = pd.concat([output_table, dataframe])
    print(dataframe.shape)
    print(output_table.shape)

['resources/predictions_all/NS-prompts_opt-125m_SST5.tsv', 'resources/predictions_all/NS-prompts_opt-350m_SST5.tsv', 'resources/predictions_all/NS-prompts_roberta-base_SST5.tsv', 'resources/predictions_all/NS-prompts_roberta-large_SST5.tsv', 'resources/predictions_all/NS-prompts_llama-7b_SST5.tsv']


  0%|                                                     | 0/5 [00:00<?, ?it/s]

(146520, 8)
(146520, 8)
(146520, 8)
(293040, 8)
(146520, 8)
(439560, 8)
(146520, 8)
(586080, 8)
(58608, 8)
(644688, 8)


## Calculate Statistics
Since there can be multiple examples for each protected group, we apply the `groupby` method to compute aggregated statistics across all the examples of each protected group.

Refer to the following function for details on how we implemented the metrics.

### Function for aggregating statistics


Below you will find a function that takes in a dataframe. This dataframe is will be automatically generated using the `pd.DataFrame.groupby`. The content of this dataframe is a subset of one particular prediction tsv for a particular protected group (e.g., the "young" group for the protected class "age"). 

The function will return a `pd.Series` (like a dictionary) mapping the following fairness metrics to their floating point values:
- Accuracy
- TPR(Positive)
- TPR(Neutral)
- TPR(Negative)
- FPR(Positive)
- FPR(Neutral)
- FPR(Negative)

Also refer to https://stackoverflow.com/a/50671617 for details on how the TPR and FPR values are computed.

In [86]:
def get_stats(dataframe: pd.DataFrame) -> pd.Series:
    """
    Input: dataframe representing all predictions for a particular
    protected group in a particular run.
    """
    output: Dict[str, float] = {}

    num_examples = len(dataframe)
    num_correct = np.sum(dataframe["y_pred"] == dataframe["y_true"])

    output["accuracy"] = num_correct / num_examples

    # See https://stackoverflow.com/a/50671617
    confusion_matrix = ConfusionMatrix(dataframe["y_true"], dataframe["y_pred"], labels=[0, 1, 2])

    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.sum() - (FP + FN + TP)

    eps = 10 ** -6

    FP = np.copy(FP) + eps
    FN = np.copy(FN) + eps
    TP = np.copy(TP) + eps
    TN = np.copy(TN) + eps

    FPR = FP / (FP + TN)
    FNR = FN / (TP + FN)
    TPR = 1 - FNR

    for label_index, label in enumerate(["negative", "neutral", "positive"]):
        output[label + "_FPR"] = FPR[label_index]
        output[label + "_TPR"] = TPR[label_index]

    return pd.Series(output)

### Calculating Per-Group Statistics

In [87]:
# output_table
analyzed_dataframe = output_table.groupby([*INDEPENDENT_VARIABLES, "run_id", "category", "group"]).apply(get_stats)
analyzed_dataframe.query('category == "grouped_race"')[["positive_TPR"]]

positive_TPR
model                    dataset num_params run_id category     group                         
llama-7b fine-tuned      SST5    7.00       r0     grouped_race african_american           0.5
                                                                american_indian            0.5
                                                                asian                      0.5
                                                                hispanic                   0.5
                                                                pacific_islander           0.5
...                                                                                        ...
roberta-large fine-tuned SST5    0.35       r9     grouped_race american_indian            0.5
                                                                asian                      0.5
                                                                hispanic                   0.5
                                                                pacific_islander           0.5
                                                                white                      0.5

[390 rows x 1 columns]

In [88]:
output_table = analyzed_dataframe.reset_index()
os.makedirs(output_folder, exist_ok=True)
output_table.to_csv(os.path.join(output_folder, "aggregated.csv"))

## Calculating Statistics

Before we begin, note that some of the protected classes include a large number of groups. You may filter on the protected groups to include in each category. If a category isn't in this dictionary, the visualizations will include all the groups under that category by default.

In [89]:
SELECTED_GROUPS = {
}

groups_to_include = []
for category in output_table["category"].unique():
    groups = SELECTED_GROUPS.get(category)
    if groups is None:
        groups = list(set(output_table.query(f'category == "{category}"')["group"]))

    groups_to_include.extend(groups)

print("Groups selected:", ", ".join(groups_to_include))
output_table = output_table.query("group == @groups_to_include")

Groups selected: asian, pacific_islander, african_american, hispanic, white, american_indian


### "Gap": How each group deviates from the category mean
Below, we will calculate how far each group deviates from the mean of that category. For example, how far does the accuracy on examples mentioning the "young" group deviate from the mean of all examples of the "age" category?

If there are more than one runs for each model, we would calculate gap metrics for each run separately. 

We use a for loop to add the following metrics of interest to our list:
- Accuracy
- False Positive Rates
    - FPR("Negative")
    - FPR("Neutral")
    - FPR("Positive")
- True Positive Rates
    - TPR("Negative")
    - TPR("Neutral")
    - TPR("Positive")

In [90]:
metrics = ["accuracy"]
for label in ["negative", "neutral", "positive"]:
    metrics.append(label + "_FPR")
    metrics.append(label + "_TPR")

gap_metrics = []
for metric in metrics:
    gap_metrics.append(f"{metric}_gap")

print(gap_metrics)

['accuracy_gap', 'negative_FPR_gap', 'negative_TPR_gap', 'neutral_FPR_gap', 'neutral_TPR_gap', 'positive_FPR_gap', 'positive_TPR_gap']


In [91]:
# Calculate gap for each metric and
# save the results in a column named f"{metrics}_gap".
for metric, gap_metric in zip(metrics, gap_metrics):
    # Calculate mean for each metric, group, and run/seed.
    group_by = [*INDEPENDENT_VARIABLES, "category", "run_id"]
    grouped = output_table[[*INDEPENDENT_VARIABLES, "category", "run_id", metric]].groupby(group_by)
    mean = grouped.transform("mean")

    # Broadcast the mean values across the table
    # to find the gap for each entry.
    normalized_values = output_table[metric] - mean[metric]
    output_table[gap_metric] = normalized_values

### Confidence Interval Calculations

In the previous step, we've found the gaps for each (metric, category, group, model, run/seed). If we have more than one runs/seeds for each model, we can aggregate the results to find a confidence interval for each gap variable.

To do so, we will aggregate the previous table (metric, category, group, model, run/seed) along the run/seed axis. The result will be a table with indices (metric, category, group, model).

In [92]:
# Create a (metric, group) placeholder table for storing the output values.
# Unlike in the previous step, "run_id" isn't part of groupby,
# and this axis will be squeezed in the result.
stats_table = (
    output_table[[*INDEPENDENT_VARIABLES, "category", "group", "accuracy"]]
    .groupby([*INDEPENDENT_VARIABLES, "category", "group"])
    .agg(["mean"])
)

In [93]:
for metric, gap_metric in zip(metrics, gap_metrics):
    group_by = [*INDEPENDENT_VARIABLES, "category"]
    grouped = output_table[[*INDEPENDENT_VARIABLES, "category", metric]].groupby(group_by)
    group_mean = grouped.transform("mean")
    normalized_values = output_table[metric] - group_mean[metric]
    output_table[gap_metric] = normalized_values

    # See https://stackoverflow.com/a/53522680
    # Calculate mean and stdev for each (metric, category, group, model).
    stats = (
        output_table[[*INDEPENDENT_VARIABLES, "category", "group", gap_metric]]
        .groupby([*INDEPENDENT_VARIABLES, "category", "group"])
        .agg(["mean", "count", "std"])
    )

    mean_values = []
    ci_width = []
    lower_values = []
    upper_values = []
    hypothesis_selected = []

    # Calculate 95% Confidence interval for each
    # (metric, category, group, model).
    for index in stats.index:
        mean, n, stdev = stats.loc[index]  # type: ignore
        lower = mean - 1.96 * stdev / sqrt(n)
        upper = mean + 1.96 * stdev / sqrt(n)

        mean_values.append(mean)
        ci_width.append(1.96 * stdev / sqrt(n))
        lower_values.append(lower)
        upper_values.append(upper)

        if lower > 0:
            hypothesis_selected.append(1)
        elif upper < 0:
            hypothesis_selected.append(-1)
        else:
            hypothesis_selected.append(0)

    stats_table[gap_metric] = mean_values
    stats_table[gap_metric + "_width"] = ci_width
    stats_table[gap_metric + "_lower"] = lower_values
    stats_table[gap_metric + "_upper"] = upper_values

stats_table = stats_table.reset_index()

## Plotting

### Compare between Groups of each category 
E.g. all groups in the "age" category

In [94]:
datasets = stats_table["dataset"].unique()
models = ["(all)", *(stats_table["model"].unique())]
categories = stats_table["category"].unique()

##### Interactive Visualization Utilities

In [95]:
model_selector = widgets.Dropdown(options=models, value=models[0], description="Model:")
category_selector = widgets.Dropdown(options=categories, value=categories[0], description="Category:")
metric_selector = widgets.Dropdown(options=metrics, value=metrics[0], description="Metric:")

In [96]:
def visualize_fn(button: Any) -> None:
    """
    Function to be called when the button is pressed.
    This function clears previous output and any widget
    that has already been rendered. To add these widgets
    back (including the button) the "button" parameter
    needs to be supplied from the button action.
    """
    clear_output()

    display(model_selector)
    display(category_selector)
    display(metric_selector)
    display(button)

    model = model_selector.value
    category: str = category_selector.value  # type: ignore
    metric: str = metric_selector.value  # type: ignore

    metric += "_gap"

    if model == "(all)":
        filtered_table = stats_table[(stats_table["category"] == category)]
    else:
        filtered_table = stats_table[(stats_table["category"] == category) & (stats_table["model"] == model)]

    if len(filtered_table) == 0:
        print("None of the runs in the table matched the selections.")
        return

    # Removing the "grouped" from grouped_{category_name}
    category_name = category.split("_")[-1]
    title = f"Gap for {category_name.title()}"

    # Generate scatterplot with confidence interval.
    fig = px.scatter(
        filtered_table,
        x="group",
        facet_col="group",
        y=metric,
        color="model",
        error_y=metric + "_width",
        labels={
            "group": "Protected Group",
            gap_metric: format_metric_name(gap_metric),
            "model": "LM",
        },
        title=title,
        height=600,
        
    )

    # Hide redundant x axis labels.
    fig.update_xaxes(matches=None, tickangle=90)
    if metric == "negative_FPR":
        fig.update_layout(yaxis_range=[-0.1,0.1])
    fig.for_each_xaxis(lambda x: x.update(title=""))
    fig.for_each_annotation(lambda a: a.update(text=""))

    fig = fig.update_layout(
        scattermode="group",
        font_color="black",
        font_size=18,
        title_x=0.5,
    )

    plot_output_path = plot_dir + metric + ".png"
    fig.write_image(plot_output_path, scale=5)
    with open(plot_output_path, "rb") as img_file:
        image_widget = widgets.Image(value=img_file.read(), format="png", width=600)
        display(image_widget)

##### Interative Visualization

In [ ]:
button = widgets.Button(description="Visualize")
button.on_click(visualize_fn)
visualize_fn(button)

Dropdown(description='Model:', options=('(all)', 'llama-7b fine-tuned', 'opt-125m fine-tuned', 'opt-350m fine-…

Dropdown(description='Category:', options=('grouped_race',), value='grouped_race')

Dropdown(description='Metric:', index=5, options=('accuracy', 'negative_FPR', 'negative_TPR', 'neutral_FPR', '…

Button(description='Visualize', style=ButtonStyle())

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\r\xac\x00\x00\x0b\xb8\x08\x06\x00\x00\x00\xb7\xe5\x1…

### Compare between models and datasets on the same protected group 
E.g. "young" people of the "age" category.


##### Interative Visualization Utilities

In [98]:
unique_groups = stats_table["group"].unique()
model_selector = widgets.Dropdown(options=models, value=models[0], description="Model:")
group_selector = widgets.Dropdown(options=unique_groups, value=unique_groups[0], description="Group:")
metric_selector = widgets.Dropdown(options=metrics, value=metrics[0], description="Metric:")

In [99]:
def visualize_fn_2(button: Any) -> None:
    clear_output()
    display(model_selector)
    display(group_selector)
    display(metric_selector)
    display(button)

    model = model_selector.value
    group: str = group_selector.value  # type: ignore
    metric: str = metric_selector.value  # type: ignore

    gap_metric = metric + "_gap"

    if model == "(all)":
        filtered_table = stats_table[(stats_table["dataset"] != "")]
    else:
        filtered_table = stats_table[(stats_table["model"] == model) & (stats_table["dataset"] != "")]

    group_filtered_table = filtered_table[filtered_table["group"] == group]

    if len(group_filtered_table) == 0:
        print("None of the runs in the table matched the selections.")
        return

    title = f"Gap for {group.title()} "

    fig = px.scatter(
        group_filtered_table,
        x="dataset",
        facet_col="dataset",
        y=gap_metric,
        color="model",
        error_y=gap_metric + "_width",
        labels={
            "group": "Protected Group",
            gap_metric: format_metric_name(gap_metric),
            "model": "LM",
            **{dataset: format_metric_name(dataset) for dataset in filtered_table["dataset"].unique()},
        },
        title=title,
        height=600,
    )

    fig.update_xaxes(matches=None, tickangle=90)
    fig.for_each_xaxis(lambda x: x.update(title=""))
    fig.for_each_annotation(lambda a: a.update(text=""))

    fig = fig.update_layout(
        scattermode="group",
        font_color="black",
        font_size=18,
        title_x=0.5,
    )

    plot_output_path = "plot.png"
    fig.write_image(plot_output_path, scale=5)
    with open(plot_output_path, "rb") as img_file:
        image_widget = widgets.Image(value=img_file.read(), format="png", width=600)
        display(image_widget)

##### Interative Visualization

In [100]:
button = widgets.Button(description="Visualize")
button.on_click(visualize_fn_2)
visualize_fn_2(button)

Dropdown(description='Model:', options=('(all)', 'llama-7b fine-tuned', 'opt-125m fine-tuned', 'opt-350m fine-…

Dropdown(description='Group:', options=('african_american', 'american_indian', 'asian', 'hispanic', 'pacific_i…

Dropdown(description='Metric:', options=('accuracy', 'negative_FPR', 'negative_TPR', 'neutral_FPR', 'neutral_T…

Button(description='Visualize', style=ButtonStyle())

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\r\xac\x00\x00\x0b\xb8\x08\x06\x00\x00\x00\xb7\xe5\x1…